In [1]:
import pandas as pd

In [2]:
EMP=pd.read_csv("https://raw.githubusercontent.com/kilianguillon/Projet_statapp/main/data/EMP_deplacements_Charme.csv",
                sep=";", encoding='latin-1')
EMP.head()

,IDENT_IND,IDENT_MEN,MVEH,POND_JOUR,MOTPREC,MMOTIFDES,MSTATION,Motif,Stationnement,Lieu_Depart,...,MVEHEXT,INSTANT_DEPART,INSTANT_ARRIVEE,DISTANCE,KVAQUIV,TRAVAILLE,Segment_detail,IDENT_V,Segment,num_dep_V
0,110000011400001,1100000114000,1,"9891,108772",1.1,2.1,5,Se rendre dans une grande surface ou un centre...,Emplacement dans un parking public ou commerci...,Domicile,...,NaN,2029-01-03 14:15:00,2029-01-03 14:20:00,"2,2",1.0,2,Particulier,110000011400001_1_Particulier,Particulier,1
1,110000011400001,1100000114000,1,"9891,108772",2.1,1.1,4,Retour au domicile,"Emplacement privé, gratuit (y compris domicile)",Parking,...,NaN,2029-01-03 14:45:00,2029-01-03 14:50:00,"2,2",1.0,2,Particulier,110000011400001_1_Particulier,Particulier,2
2,110000011500001,1100000115000,2,"5275,2959481",1.1,1.4,6,"Étudier (école, lycée, université)",Emplacement dans un parking public ou commerci...,Domicile,...,NaN,2029-01-05 07:45:00,2029-01-05 08:45:00,28,1.0,2,Particulier,110000011500001_2_Particulier,Particulier,1
3,110000011500001,1100000115000,2,"5275,2959481",1.4,1.1,4,Retour au domicile,"Emplacement privé, gratuit (y compris domicile)",Parking,...,NaN,2029-01-05 10:30:00,2029-01-05 11:00:00,28,1.0,2,Particulier,110000011500001_2_Particulier,Particulier,2
4,110000013500002,1100000135000,1,"24058,724023",1.1,9.1,4,Travailler dans son lieu fixe et habituel,"Emplacement privé, gratuit (y compris domicile)",Domicile,...,NaN,2029-01-03 15:00:00,2029-01-03 15:15:00,"5,3214103219",1.0,1,Particulier,110000013500002_1_Particulier,Particulier,1


In [3]:
#Création d'une base ne prenant que les premiers trajets
EMP_prems = EMP[EMP["num_dep_V"]==1]

In [4]:
EMP_prems["Motif"].value_counts() #Motif du déplacement (pas de pondération)

Motif
Travailler dans son lieu fixe et habituel                                                                                                                                    1838
Se rendre dans une grande surface ou un centre commercial (y compris boutiques et services)                                                                                  1111
Se rendre dans un centre de proximité, petit commerce, supérette, boutiques, services (banque, cordonnier.) (hors centre commercial)                                          891
Visite à la famille                                                                                                                                                           471
Accompagner quelqu'un à un autre endroit                                                                                                                                      408
Soins médicaux ou personnels (médecin, coiffeur.)                                                       

In [5]:
#Arrivée du déplacement pas de pond
EMP_prems["Lieu_Arrivee"].value_counts()

Lieu_Arrivee
Rue           2577
Parking       2486
Entreprise    1441
Domicile       570
Sans           275
Name: count, dtype: int64

In [6]:
value_motif = EMP_prems["Motif"].unique()
value_arrivee = EMP_prems["Lieu_Arrivee"].unique()
#indique les outputs possibles de ces variables (à ajouter dans la fonction)

In [ ]:
def count_occ_pond(data, nom_var, nom_pond):
    value = data[nom_var].unique()
    tabl=[value,[float(0)] * len(value)]
    ind=0
    for elem in data[nom_var]: 
        k=0
        for possible_output in tabl[0] :
            if elem == possible_output :
                tabl[k][1] += float(data[nom_pond][ind].replace(',', '.'))
            k += 1
        ind += 1
    return pd.DataFrame(tabl)

In [ ]:
count_occ_pond(EMP_prems, "Motif", "POND_JOUR")